In [ ]:
import tensorflow as tf
from psutil import virtual_memory

# Check GPU
gpu_info = tf.config.list_physical_devices('GPU')
print(f"GPU Info: {gpu_info}")

# Check RAM
ram_info = virtual_memory()
print(f"Total RAM: {ram_info.total / (1024**3)} GB")

In [ ]:
!nvidia-smi -L

In [53]:
# Tell llama.cpp to use cuBLAS (fast CUDA kernels)
%env LLAMA_CUBLAS=1

# How many transformer layers to keep on-GPU.
# 35 is a sweet-spot for 8-B models on a 16 GB T4.
%env OLLAMA_NUM_GPU_LAYERS=35

# Optional: cap the model’s maximum output length (keeps responses snappy)
%env OLLAMA_MAX_PREDICT=128

env: LLAMA_CUBLAS=1
env: OLLAMA_NUM_GPU_LAYERS=35
env: OLLAMA_MAX_PREDICT=128


In [54]:
!pip install colab-xterm #https://pypi.org/project/colab-xterm/
%load_ext colabxterm

!pip install langchain -qqq
!pip install langchain_community -qqq
!pip install faiss-cpu -qqq
!pip install sentence-transformers -qqq
!pip install firebase-admin -qqq
# 🛠️ Install all libraries needed by the API layer and the bot itself
!pip install flask flask_cors pyngrok






The colabxterm extension is already loaded. To reload it, use:
  %reload_ext colabxterm


In [56]:
!sudo apt-get update -qq
!sudo apt-get install -y -qq pciutils lshw   # provides lspci / lshw


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [57]:
!sudo rm -f /usr/local/bin/ollama
!sudo rm -rf /usr/local/lib/ollama
!pkill -f ollama || true     # ignore “no process” errors


^C


In [58]:
!curl -fsSL https://ollama.com/install.sh | sh
!ollama --version            # should end with “(CUDA)”

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [59]:
%env LLAMA_CUBLAS=1           # use fast cuBLAS kernels
%env OLLAMA_NUM_GPU_LAYERS=35 # good value for 8-B on a 16 GB T4
%env OLLAMA_MAX_PREDICT=128   # optional response-length cap


env: LLAMA_CUBLAS=1           # use fast cuBLAS kernels
env: OLLAMA_NUM_GPU_LAYERS=35 # good value for 8-B on a 16 GB T4
env: OLLAMA_MAX_PREDICT=128   # optional response-length cap


In [60]:
!ollama serve > /dev/null 2>&1 &


In [61]:
!ollama pull llama3:instruct      # 3-B model, fully on-GPU

from langchain_community.llms import Ollama
import typing_extensions
import time

llm = Ollama(model="llama3:instruct")

t0 = time.time()
print("Model reply →", llm.invoke("ping"))
print("⏱️  Elapsed:", round(time.time() - t0, 2), "s")



Model reply → You typed "ping"!

Ping is a network troubleshooting tool that sends an ICMP (Internet Control Message Protocol) echo request packet to a destination and measures the time it takes for the packet to return. This allows you to test whether a particular host or IP address is reachable, as well as measure the latency between your device and the target.

Here's a basic example of how to use ping in Windows:

1. Open Command Prompt (Windows key + R, type "cmd" and press Enter).
2. Type `ping [destination]` (replace `[destination]` with the IP address or hostname you want to test).

Example: `ping google.com`

3. Press Enter.

The output will show information about the ping request, including:

* The timestamp when the packet was sent
* The destination IP address and name
* The round-trip time (RTT) in milliseconds
* Whether the packet was received successfully

If you receive a response with a low RTT (<100 ms), it indicates that the connection is working well. Higher RTTs ma

In [ ]:
import firebase_admin
from firebase_admin import credentials, db
import json
from pathlib import Path

# Upload your Firebase key file to Colab files first
# FIREBASE_KEY_PATH = "foodbot-senior-firebase-adminsdk-fbsvc-5d0630eac0.json"  # Update path as needed



# Cell 4: Upload Firebase credentials
from google.colab import files
print("Please upload your Firebase Admin SDK JSON file (foodbot-1-firebase-adminsdk-fbsvc-7a944f4dc9.json):")
uploaded = files.upload()
FIREBASE_KEY_PATH = next(iter(uploaded.keys()))
print(f"Uploaded: {FIREBASE_KEY_PATH}")


# Only initialize if not already
if not firebase_admin._apps:
    cred = credentials.Certificate(FIREBASE_KEY_PATH)
    firebase_admin.initialize_app(cred, {"databaseURL": FIREBASE_URL})

db_ref = db.reference()
user_ref = db_ref.child("users").child(TEST_USER_ID)
restaurant_ref = db_ref.child("restaurants")

# Setup data paths
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)
MENU_JSON_PATH = DATA_DIR / "menu_faq.json"
SUMMARY_TXT_PATH = DATA_DIR / "generated_dish_summaries2.txt"

Please upload your Firebase Admin SDK JSON file (foodbot-1-firebase-adminsdk-fbsvc-7a944f4dc9.json):


Saving foodbot-senior-firebase-adminsdk-fbsvc-5d0630eac0.json to foodbot-senior-firebase-adminsdk-fbsvc-5d0630eac0 (1).json
Uploaded: foodbot-senior-firebase-adminsdk-fbsvc-5d0630eac0 (1).json


In [63]:


# run the server in the background; redirect output to a log file
!ollama serve > /content/ollama.log 2>&1 &


In [64]:
import time
time.sleep(3)

In [65]:

from langchain_community.llms import Ollama

# Initialize Ollama with llama3 (make sure you've run ollama serve & ollama pull llama3 in terminal)
llm = Ollama(model="llama3:instruct")

# Test the connection
test_response = llm.invoke("Hello, are you working?")
print(f"Ollama test response: {test_response}")

Ollama test response: Hello! I'm an AI, so I don't have a traditional job or work schedule like humans do. However, I am always "on the clock" and ready to assist with any questions or tasks you may have.

I exist solely to provide information, answer questions, and help users like you with their queries. My training data is constantly being updated and expanded, so I'm always learning and improving my abilities.

So, in a sense, I am working 24/7 to provide the best possible assistance and responses to those who interact with me!


In [66]:

import os
import json
import re

def create_restaurant_summaries(json_path, output_dir="restaurant_data"):
    print(f"🚀 Starting restaurant summary creation...")

    # Check if JSON file exists
    if not json_path.exists():
        raise FileNotFoundError(f"❌ Menu JSON file not found at {json_path}. Please upload your menu_faq.json file to Colab.")

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    print(f"📁 Created output directory: {output_dir}")

    with open(json_path, "r", encoding='utf-8') as f:
        full_data = json.load(f)

    restaurant_files = {}
    restaurants = full_data.get("restaurants", {})
    print(f"🏪 Processing {len(restaurants)} restaurants...")

    for restaurant_id, restaurant_data in restaurants.items():
        restaurant_name = restaurant_data.get("name", restaurant_id).strip()

        location = restaurant_data.get("location", "Unknown Location")
        food_type = restaurant_data.get("foodType", "Cuisine not specified")
        menu = restaurant_data.get("menu", {})

        print(f"\n🔄 Processing: {restaurant_name} (ID: {restaurant_id})")
        print(f"   📍 Location: {location}")
        print(f"   🍽️ Food Type: {food_type}")
        print(f"   📋 Menu items: {len(menu)}")

        if not menu:
            print(f"   ⚠️ Skipping {restaurant_name} - no menu items found")
            continue

        summaries = []
        restaurant_header = (
            f"Restaurant: {restaurant_name}\n"
            f"Location: {location}\n"
            f"Cuisine Type: {food_type}\n"
            f"{'='*50}\n"
        )
        summaries.append(restaurant_header)

        dish_count = 0
        for dish_id, dish_data in menu.items():
            name       = dish_data.get("name", "Unnamed Dish")
            ingredients = ", ".join(dish_data.get("ingredients", []))
            allergens   = ", ".join(dish_data.get("allergens", []))
            dietary     = ", ".join(dish_data.get("dietary", []))
            calories    = dish_data.get("calories", "N/A")
            price       = dish_data.get("price", "N/A")
            order_count = dish_data.get("orderCount", "N/A")

            print(f"   📝 Processing dish: {name}")

            prompt = f"""Write EXACTLY ONE sentence describing this dish. You MUST include ALL available information in this exact order:

"The [DISH NAME] at [RESTAURANT NAME] is [description with ingredients], contains [allergens if any], suitable for [dietary restrictions if any], priced at $[PRICE], containing [CALORIES] calories, and has been ordered [ORDER COUNT] times."

Available details:
- Dish: {name}
- Restaurant: {restaurant_name}
- Ingredients: {ingredients}
- Price: ${price}
- Calories: {calories}
- Allergens: {allergens if allergens else "no known allergens"}
- Dietary: {dietary if dietary else "no specific dietary restrictions"}
- Order Count: {order_count}

RULES:
1. Include ALL details above in the sentence
2. If allergens is empty, write "contains no known allergens"
3. If dietary is empty, write "suitable for no specific dietary restrictions"
4. Always include the price, calories, and order count
5. Write only ONE sentence, no extra text"""

            try:
                # 🔹 Ask the model
                summary_raw = llm.invoke(prompt).strip()

                # 🔹 Remove any leading boiler-plate like “Here is the exact sentence:”
                summary_raw = re.sub(
                    r'^\s*["\']?\s*(here[\w\s]*sentence[:\-]?)\s*["\']?\s*',
                    '',
                    summary_raw,
                    flags=re.I
                ).strip()

                # 🔹 Take the first full sentence
                clean_summary = re.split(r'(?<=[.!?])\s+', summary_raw, maxsplit=1)[0].strip()

                # 🔹 Strip wrapping quotes
                clean_summary = clean_summary.strip(' "\'')

                summaries.append(f"Dish: {name}\n{clean_summary}\n")
                print(f"   ✅ Generated summary for: {name}")
                dish_count += 1

            except Exception as e:
                print(f"   ❌ Error with {name}: {e}")

        # Save restaurant file
        safe_filename = "".join(c for c in restaurant_name if c.isalnum() or c in (' ', '-', '_')).strip()
        restaurant_file = os.path.join(output_dir, f"{safe_filename}.txt")

        try:
            with open(restaurant_file, "w", encoding='utf-8') as f:
                f.write("\n".join(summaries))

            restaurant_files[restaurant_name] = restaurant_file
            print(f"   💾 Saved {restaurant_name} to {restaurant_file}")
            print(f"   📊 Processed {dish_count} dishes successfully")

        except Exception as e:
            print(f"   ❌ Error saving file for {restaurant_name}: {e}")

    print(f"\n🎉 Completed! Created {len(restaurant_files)} restaurant files:")
    for name, file_path in restaurant_files.items():
        print(f"   📄 {name}: {file_path}")

    return restaurant_files


# 🔥 Execute the function
print("🔥 Running create_restaurant_summaries...")
try:
    restaurant_files = create_restaurant_summaries(MENU_JSON_PATH)
    print(f"\n✅ SUCCESS: Created {len(restaurant_files)} restaurant files")

except Exception as e:
    print(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()


🔥 Running create_restaurant_summaries...
🚀 Starting restaurant summary creation...
📁 Created output directory: restaurant_data
🏪 Processing 3 restaurants...

🔄 Processing: Italian Bistro (ID: restaurant1)
   📍 Location: NewYork
   🍽️ Food Type: Italian
   📋 Menu items: 9
   📝 Processing dish: Spaghetti Bolognese
   ✅ Generated summary for: Spaghetti Bolognese
   📝 Processing dish: Margherita Pizza
   ✅ Generated summary for: Margherita Pizza
   📝 Processing dish: Lasagna
   ✅ Generated summary for: Lasagna
   📝 Processing dish: Fettuccine Alfredo
   ✅ Generated summary for: Fettuccine Alfredo
   📝 Processing dish: Penne Arrabbiata
   ✅ Generated summary for: Penne Arrabbiata
   📝 Processing dish: Risotto
   ✅ Generated summary for: Risotto
   📝 Processing dish: Caprese Salad
   ✅ Generated summary for: Caprese Salad
   📝 Processing dish: Bruschetta
   ✅ Generated summary for: Bruschetta
   📝 Processing dish: Ravioli
   ✅ Generated summary for: Ravioli
   💾 Saved Italian Bistro to resta

In [67]:
from datetime import datetime, timezone

def start_new_chat(user_id: str) -> str:
    """Create a new chat node and return its key."""
    chat_ref = (db_ref.child("users")
                       .child(user_id)
                       .child("chats")
                       .push({            # push() ⇒ unique key
                           "started_at": datetime.now(timezone.utc).isoformat()
                       }))
    return chat_ref.key                   # <chat_id>

def save_message(user_id: str, chat_id: str,
                 sender: str, text: str) -> None:
    (db_ref.child("users")
           .child(user_id)
           .child("chats")
           .child(chat_id)
           .child("messages")
           .push({                       # one element per message
               "sender": sender,         # "user" | "bot"
               "text": text,
               "ts": datetime.now(timezone.utc).isoformat()
           }))

def fetch_recent_history(user_id: str, chat_id: str,
                         limit:int=20) -> list[dict]:
    """Return the last N messages (oldest→newest)."""
    snap = (db_ref.child("users")
                   .child(user_id)
                   .child("chats")
                   .child(chat_id)
                   .child("messages")
                   .order_by_child("ts")   # sort server-side
                   .limit_to_last(limit)
                   .get() or {})
    # snap is a dict keyed by msg_id; convert & sort just in case
    msgs = sorted(snap.values(), key=lambda m: m["ts"])
    return msgs


In [68]:

from langchain_community.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

# Only proceed if we have restaurant files
if not restaurant_files:
    print("❌ No restaurant files found. Please ensure menu_faq.json is uploaded and Cell 6 runs successfully.")
else:
    # Initialize embeddings
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

    # Store all restaurant vectorstores
    restaurant_vectorstores = {}

    def create_restaurant_vectorstore(restaurant_name, file_path):
        """Create a FAISS vectorstore for a specific restaurant"""
        with open(file_path, "r", encoding='utf-8') as f:
            text_data = f.read()

        # Split into chunks but keep restaurant context
        text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=50)
        chunks = text_splitter.split_text(text_data)

        docs = []
        for i, chunk in enumerate(chunks):
            docs.append(Document(
                page_content=chunk,
                metadata={
                    "restaurant": restaurant_name,
                    "chunk_id": i,
                    "source": file_path
                }
            ))

        # Create FAISS vectorstore for this restaurant
        vectorstore = FAISS.from_documents(docs, embeddings)
        return vectorstore

    # Create vectorstore for each restaurant
    for restaurant_name, file_path in restaurant_files.items():
        print(f"🔍 Creating vectorstore for {restaurant_name}...")
        restaurant_vectorstores[restaurant_name] = create_restaurant_vectorstore(restaurant_name, file_path)
        print(f"✅ {restaurant_name} vectorstore ready")

    print(f"\n🎯 Created vectorstores for {len(restaurant_vectorstores)} restaurants:")
    for name in restaurant_vectorstores.keys():
        print(f"  - {name}")

🔍 Creating vectorstore for Italian Bistro...
✅ Italian Bistro vectorstore ready
🔍 Creating vectorstore for American Grill...
✅ American Grill vectorstore ready
🔍 Creating vectorstore for McDonald's...
✅ McDonald's vectorstore ready

🎯 Created vectorstores for 3 restaurants:
  - Italian Bistro
  - American Grill
  - McDonald's


In [69]:
# ──────────────────────────────────────────────────────────────
#  Firebase helpers + FoodBot core (auto-creates chat_id)
# ──────────────────────────────────────────────────────────────
from datetime import datetime, timezone

TEST_USER_ID = "xHi2Sak4Z8OYn9Vd1NSMwGfk6Sj1"  # update if needed

def start_new_chat(user_id: str) -> str:
    chat_ref = (db_ref.child("users")
                       .child(user_id)
                       .child("chats")
                       .push({
                           "started_at": datetime.now(timezone.utc).isoformat()
                       }))
    return chat_ref.key

def save_message(user_id: str, chat_id: str, sender: str, text: str) -> None:
    (db_ref.child("users")
           .child(user_id)
           .child("chats")
           .child(chat_id)
           .child("messages")
           .push({
               "sender": sender,
               "text": text,
               "ts": datetime.now(timezone.utc).isoformat()
           }))

def fetch_recent_history(user_id: str, chat_id: str, limit: int = 20) -> list:
    snap = (db_ref.child("users")
                   .child(user_id)
                   .child("chats")
                   .child(chat_id)
                   .child("messages")
                   .order_by_child("ts")
                   .limit_to_last(limit)
                   .get() or {})
    return sorted(snap.values(), key=lambda m: m["ts"])

def get_user_context(user_id=TEST_USER_ID):
    try:
        return db_ref.child("users").child(user_id).get() or {}
    except Exception as e:
        print(f"Error getting user context: {e}")
        return {}

# ── ask_foodbot now creates a chat automatically if chat_id is None ──
def ask_foodbot(query: str,
                chat_id: str | None = None,
                user_id: str = TEST_USER_ID,
                history_window: int = 20) -> str:
    """
    Answer any question by searching *all* restaurant vector-stores.
    If `chat_id` is None, a brand-new chat node is created automatically.
    """
    # 0. Create chat on-the-fly if needed
    if chat_id is None:
        chat_id = start_new_chat(user_id)

    if not restaurant_vectorstores:
        return ("❌ No restaurant data available. "
                "Upload menu_faq.json and run the setup cells first.")

    # 1. Log user message
    save_message(user_id, chat_id, "user", query)

    # 2. Keep Ollama alive
    try:
        llm.invoke("ping")
    except Exception:
        print("🔄 Restarting Ollama…")
        import subprocess, time, os
        subprocess.run(["pkill", "-f", "ollama"], capture_output=True)
        time.sleep(2)
        subprocess.Popen(["ollama", "serve"],
                         stdout=subprocess.DEVNULL,
                         stderr=subprocess.DEVNULL)
        time.sleep(5)

    # 3. Retrieve context from all restaurants
    hits = []
    for rest_name, vs in restaurant_vectorstores.items():
        for h in vs.similarity_search(query, k=6):
            h.metadata["restaurant"] = rest_name
            hits.append(h)
    context = "\n\n".join(doc.page_content for doc in hits[:8])
    restaurants_searched = list(restaurant_vectorstores.keys())

    # 4. Build threaded history
    history = fetch_recent_history(user_id, chat_id, history_window)
    threaded_context = "\n".join(f"{m['sender']}: {m['text']}" for m in history
                                 if m.get("text"))

    # 5. User profile
    user = get_user_context(user_id)
    allergies   = ", ".join(user.get("allergies", []))   or "none"
    preferences = ", ".join(user.get("preferences", [])) or "none"

    prompt = f"""
You are FoodBot, a friendly restaurant assistant.

User allergies: {allergies}
User preferences: {preferences}

Restaurants searched: {', '.join(restaurants_searched)}

Menu information:
{context}

Conversation so far:
{threaded_context}

User question: {query}

Instructions:
- Mrovide short, clear and consise answers.
- Only provide the information everything the user wants.
- If you don’t know, say so politely.
-

Answer:
"""

    try:
        response = llm.invoke(prompt).strip()
        save_message(user_id, chat_id, "bot", response)
        return response
    except Exception as e:
        err = f"Sorry, I encountered an error: {e}"
        save_message(user_id, chat_id, "bot", err)
        return err

# ── Interactive loop (continues using one chat per session) ──
def chat_with_foodbot(resume_chat_id: str | None = None):
    """
    • If resume_chat_id is None, start a brand-new chat for this session.
    • Otherwise continue the specified chat.
    """
    chat_id = resume_chat_id or start_new_chat(TEST_USER_ID)
    print(f"💬 Chat id = {chat_id[:8]}…  (type 'exit' to quit)")

    while True:
        query = input("🧑 You: ").strip()
        if query.lower() in {"exit", "quit"}:
            print("👋 Goodbye!")
            break

        print("-" * 60)
        answer = ask_foodbot(query, chat_id)
        print(f"🤖 FoodBot: {answer}")
        print("-" * 60)


In [70]:
# Cell : install & import the HTTP layer
!pip install -q flask flask-cors pyngrok

from flask import Flask, request, jsonify
from flask_cors import CORS
from threading import Thread
from pyngrok import ngrok


In [ ]:
# --- DIAGNOSTIC: check what token pyngrok sees ----------------------------
import os, json, subprocess, sys
from pyngrok import ngrok, conf, process



# 0️⃣ wipe any conflicting env var or config file for this session
os.environ.pop("NGROK_AUTH_TOKEN", None)     # comment out if you prefer env-var method
conf.PyngrokConfig.config_path = None        # ignore ~/.ngrok2/ngrok.yml

# 1️⃣ pass the token directly
ngrok.set_auth_token(REAL_TOKEN)

# 2️⃣ confirm what token pyngrok will send
print("pyngrok will send token:", conf.get_default().auth_token[:8] + "…")
assert conf.get_default().auth_token == REAL_TOKEN, "Token mismatch!"

# 3️⃣ quick sanity: what ngrok binary is in PATH?
print("ngrok version:", subprocess.check_output(["ngrok", "version"]).decode().strip())

# 4️⃣ open a dummy tunnel on an unused port (e.g., 9999) just to test auth
try:
    test_url = ngrok.connect(9999).public_url
    print("✅ auth OK, test tunnel:", test_url)
    ngrok.disconnect(test_url)   # close it
except process.PyngrokNgrokError as e:
    print("❌ still failing ->", e)
    sys.exit(1)


TypeError: 'NoneType' object is not subscriptable

In [ ]:
def test_restaurant_detection():
    if not restaurant_vectorstores:
        print("❌ No restaurant data available for testing. Please upload menu_faq.json first.")
        return

    test_queries = [
    "How much protein is in Grilled Salmon?",
    "What's the protein content in a Cheeseburger?",
    "How many calories are in a Big Mac?",
    "How much fat is in Spaghetti Bolognese?",
    "What is the price of Grilled Salmon?",
    "What ingredients are in a Grilled Salmon?",
    "What are the ingredients in Spaghetti Bolognese?",
    "How many calories are in the Margherita Pizza?",
    "What is the price of Lasagna?",
    "Is the Fettuccine Alfredo vegetarian?",
    "Does Penne Arrabbiata contain gluten?",
    "What allergens are in Risotto?",
    "How many times has Caprese Salad been ordered?",
    "Is Bruschetta vegan?",
    "What are the allergens in Ravioli?",
    "How many calories are in the Cheeseburger?",
    "What is the price of the Grilled Chicken Sandwich?",
    "Does Hot Dog have any dietary labels?",
    "What allergens are in BBQ Ribs?",
    "How many calories are in Grilled Salmon?",
    "Is Caesar Salad vegetarian?",
    "How much does Chicken Tenders cost?",
    "What allergens are in Mac and Cheese?",
    "What dietary restriction does Buffalo Wings support?",
    "How many calories are in Big Mac?",
    "What is the price of Spicy McChicken?",
    "Does McChicken have a Halal label?",
    "What ingredients are in Filet-O-Fish?",
    "Are Chicken McNuggets Halal?",
    "What is the calorie count of French Fries?",
    "What dietary label does Hash Browns have?",
    "How much is a Chocolate Shake?",
    "What allergens are in McFlurry?",
    "What are the ingredients of Egg McMuffin?",
    "What is the price of Sausage Biscuit?",
    "Is Apple Pie vegan?",
    "Which dish has the most calories at Italian Bistro?",
    "What is the average rating of American Grill?",
    "How many dishes at McDonald's are marked Halal?",
    "What allergens are in Grilled Chicken Sandwich?",
    "What is the calorie count of the Margherita Pizza?",
    "how much calories is in cheeseburger?",
    "What is the dietary category for Ravioli?",
    "Which dish contains buttermilk and honey mustard?",
    "how much for a Big Mac?",
    "Which restaurant offers the cheapest meal?"
    ]

    for query in test_queries:
        print(f"🧑 User: {query}")
        response = ask_foodbot(query)
        print(f"🤖 FoodBot: {response.strip()}")
        print("-" * 80)

# Run the test
# test_restaurant_detection()

In [ ]:


# -------------------------------------------------------------
#  Chat loop that logs every turn to Firebase and re-uses history
# -------------------------------------------------------------
def chat_with_foodbot(resume_chat_id: str | None = None):
    """
    Start an interactive session with FoodBot.
    • If `resume_chat_id` is None, a brand-new chat node is created.
    • Otherwise we keep adding to the existing chat.
    """
    if not restaurant_vectorstores:
        print("❌ No restaurant data available. Upload menu_faq.json and run setup cells first.")
        return

    # 1️⃣  Pick chat id (new or existing)
    if resume_chat_id:
        chat_id = resume_chat_id
        print(f"🔄 Resuming chat {chat_id[:8]}…")
    else:
        chat_id = start_new_chat(TEST_USER_ID)   # helper we defined earlier
        print(f"💬 New chat started (id={chat_id[:8]}…). Type 'exit' to quit.")

    print("🏪 Restaurants loaded:", ", ".join(restaurant_vectorstores.keys()))
    print()

    # 2️⃣  Main loop
    while True:
        query = input("🧑 You: ").strip()
        if query.lower() in {"exit", "quit"}:
            print("👋 Goodbye!")
            break

        print("-" * 60)
        print(f"🧑 User: {query}")

        try:
            # ask_foodbot now expects (query, chat_id, ...)
            response = ask_foodbot(query, chat_id)
            print(f"🤖 FoodBot: {response.strip()}")
        except Exception as e:
            print(f"❌ Error: {e}")

        print("-" * 60)

# 👉 Run this to start a **fresh** chat each session
chat_with_foodbot()

# 👉 Or pass a previous chat_id string to continue a thread
# chat_with_foodbot(resume_chat_id="-N9abc123xyz")
